In [1]:
import os
import cv2
import imghdr
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import random_split, DataLoader
from torchvision import transforms
import torch.nn.functional as F

In [2]:
# Define the Dataset class
DataDir = 'image data'
ImageEx = ['jpeg', 'jpg', 'bmp', 'png']
print(os.listdir(DataDir))
print(os.listdir(os.path.join(DataDir, 'Beyonce',)))


['Beyonce', 'JackieChan', 'Messi', 'Rihanna']
['00-story-beyonce-5-things.jpg', '01beyonce-item-mobileMasterAt3x.jpg', '0x0.jpg', '1169175.jpg', '1659416561-beyonce-AP.jpg', '200531232831-beyonce-london-2019.jpg', '220613-Beyonce-al-1031-3834b9.jpg', '619e37f3a0209c4d9ec736ae83c8090d--beyonce-hair-color-beyonce-blonde-hair.jpg', '68cd9636bbf57d460cf08a527d396557.jpeg', '960x0.jpg', 'a-birthday-shout-out-from-beyonce-complete-with-a-throwback-photo-1628162387.jpg', 'beyonc-c3-a9-accepts-the-best-dance-electronic-music-album-award-news-photo-1675652889.jpg', 'beyonc-c3-a9-attends-the-atlantis-the-royal-grand-reveal-weekend-news-photo-1674462721.jpg', 'BeyoncC3A9_at_The_Lion_King_European_Premiere_2019.png', 'Beyonce-1200-1.jpg', 'BEYONCE-2.jpg', 'beyonce-2023-billboard-1548.jpg', 'beyonce-and-jay-z-date-night-9003249182869256140851..jpg', 'Beyonce-cr-Courtesy-of-Parkwood-Entertainment-press-2022-billboard-1548.jpg', 'beyonce-fake-albums-on-spotify.jpg', 'beyonce-grammys-1675876501.jpg', 

In [3]:
for image_class in os.listdir(DataDir):
    for image in os.listdir(os.path.join(DataDir,image_class)):
        print(image)


00-story-beyonce-5-things.jpg
01beyonce-item-mobileMasterAt3x.jpg
0x0.jpg
1169175.jpg
1659416561-beyonce-AP.jpg
200531232831-beyonce-london-2019.jpg
220613-Beyonce-al-1031-3834b9.jpg
619e37f3a0209c4d9ec736ae83c8090d--beyonce-hair-color-beyonce-blonde-hair.jpg
68cd9636bbf57d460cf08a527d396557.jpeg
960x0.jpg
a-birthday-shout-out-from-beyonce-complete-with-a-throwback-photo-1628162387.jpg
beyonc-c3-a9-accepts-the-best-dance-electronic-music-album-award-news-photo-1675652889.jpg
beyonc-c3-a9-attends-the-atlantis-the-royal-grand-reveal-weekend-news-photo-1674462721.jpg
BeyoncC3A9_at_The_Lion_King_European_Premiere_2019.png
Beyonce-1200-1.jpg
BEYONCE-2.jpg
beyonce-2023-billboard-1548.jpg
beyonce-and-jay-z-date-night-9003249182869256140851..jpg
Beyonce-cr-Courtesy-of-Parkwood-Entertainment-press-2022-billboard-1548.jpg
beyonce-fake-albums-on-spotify.jpg
beyonce-grammys-1675876501.jpg
Beyonce-Grammys-2023.jpg
beyonce-ivy-park.jpg
beyonce-knowles-los-angeles-ca-august-mtv-video-music-awards-for

In [4]:
total_images = 0  # initialize counter variable
for image_class in os.listdir(DataDir):
    for image in os.listdir(os.path.join(DataDir, image_class)):
        total_images += 1  # increment counter for each file
print("Total number of images:", total_images)

Total number of images: 353


In [5]:
total_images = 0  # initialize counter variable
removed_images = 0  # initialize counter variable for removed images
for image_class in os.listdir(DataDir):
    for image in os.listdir(os.path.join(DataDir, image_class)):
        ImagePath = os.path.join(DataDir, image_class,image)
        try:
            DImage = cv2.imread(ImagePath)
            ImaS = imghdr.what(ImagePath)
            if ImaS not in ImageEx:
                print('Image not in extension list {}'.format(ImagePath))
                os.remove(ImagePath)
                removed_images += 1  # increment counter for each removed file
        except Exception as e:
            print('Issue with image {}'.format(ImagePath))
            # os.remove(ImagePath)  # Uncomment this if you want to remove the problematic images
        total_images += 1  # increment counter for each file

print("Total number of images found:", total_images)
print("Total number of images removed:", removed_images)


Total number of images found: 353
Total number of images removed: 0


In [6]:
# Define the transforms for the dataset
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [7]:
# Load the dataset
dataset = datasets.ImageFolder('Image data', transform=transform_train)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [8]:
print(train_size)

246


In [9]:
# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [10]:
print(train_loader)

In [11]:
# Define the device (GPU or CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [12]:
import torch.nn as nn
import torchvision.models as models

class ImageClassificationModule(nn.Module):
    def __init__(self):
        super(ImageClassificationModule, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 10)

    def forward(self, x):
        x = self.resnet(x)
        return x


In [13]:

# Initialize the model and move it to the device
from torchsummary import summary
model = ImageClassificationModule().to(device)

summary(model, input_size=(3, 224, 224))


C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [14]:

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [17]:
# Train the model
for epoch in range(3):
    train_loss = 0.0
    train_correct = 0
    val_loss = 0.0
    val_correct = 0

    # Training loop
    model.train()
    for inputs, labels in train_loader:\
        inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)


    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    train_correct += (predicted == labels).sum().item()

In [18]:

    # Validation loop
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

In [19]:

# Print statistics
train_loss /= len(train_loader.dataset)
train_acc = train_correct / len(train_loader.dataset)
val_loss /= len(val_loader.dataset)
val_acc = val_correct / len(val_loader.dataset)

In [20]:
print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc:    {val_acc:.4f}')


Epoch 3, Train Loss: 0.0011, Train Acc: 0.0244, Val Loss: 0.0402, Val Acc:    0.8077


In [21]:
# Test the model on the test set
test_loss = 0.0
test_correct = 0
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        test_correct += (predicted == labels).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc = test_correct / len(test_loader.dataset) * 100.0
    print(f'Test Loss: {test_loss:.6f}, Test Accuracy: {test_acc:.2f}%')


Test Loss: 0.024487, Test Accuracy: 85.19%


In [22]:
# save the model
torch.save(model.state_dict(), 'Models/pytorchmodel.pth')
